In [ ]:
!pip install requests


In [17]:
import json
from ipywidgets import Text, Button, Output
from IPython.display import display
import requests
import xml.etree.ElementTree as ET
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime, timedelta

# ファイル名設定
keyword_file = "search_keywords.json"

# PubMed API設定
api_key = "a7958d4ab8f82c1de8158c70b276d935b908"  # PubMedのAPIキー
recipient = "yasu1986m@gmail.com"  # メール受信者
sender_email = "yasu1986m@gmail.com"  # 送信者メールアドレス
sender_password = "edsu fxnd gqss liad"  # 送信者メールパスワード

# 入力ボックスとボタン
text = Text(description="キーワード:")
button = Button(description="検索実行")
output = Output()

# キーワードの保存
def save_keywords(keywords):
    with open(keyword_file, "w") as f:
        json.dump(keywords, f)

# キーワードの読み込み
def load_keywords():
    try:
        with open(keyword_file, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return None

# PubMed API検索関数
def search_pubmed(query, start_date, end_date, api_key):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    date_range = f"{start_date}:{end_date}[dp]"
    full_query = f"{query} AND {date_range}"
    params = {
        "db": "pubmed",
        "term": full_query,
        "retmode": "json",
        "sort": "date",
        "retmax": 10,
        "api_key": api_key
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    return response.json()["esearchresult"]["idlist"]

# PubMed APIで論文詳細を取得
def fetch_abstracts(ids, api_key):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        "db": "pubmed",
        "id": ",".join(ids),
        "retmode": "xml",
        "rettype": "abstract",
        "api_key": api_key
    }
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    return response.text

# XML解析関数
def parse_pubmed_data(xml_data):
    """PubMedから取得したXMLデータを解析"""
    root = ET.fromstring(xml_data)
    articles = []
    for article in root.findall(".//PubmedArticle"):
        pmid = article.find(".//PMID").text if article.find(".//PMID") is not None else "No PMID"
        title = article.find(".//ArticleTitle").text if article.find(".//ArticleTitle") is not None else "No Title"

        # First Authorとその所属を取得
        first_author = "No First Author"
        affiliation = "No Affiliation"
        first_author_element = article.find(".//Author")
        if first_author_element is not None:
            last_name = first_author_element.find("LastName")
            fore_name = first_author_element.find("ForeName")
            if last_name is not None and fore_name is not None:
                first_author = f"{fore_name.text} {last_name.text}"
            affiliation_element = first_author_element.find(".//AffiliationInfo/Affiliation")
            if affiliation_element is not None:
                affiliation = affiliation_element.text

        # 全著者リストを構築
        authors = []
        for author in article.findall(".//Author"):
            last_name = author.find("LastName")
            fore_name = author.find("ForeName")
            if last_name is not None and fore_name is not None:
                authors.append(f"{fore_name.text} {last_name.text}")
        author_list = ", ".join(authors) if authors else "No Authors"

        journal = article.find(".//Title").text if article.find(".//Title") is not None else "No Journal"
        pub_date = article.find(".//PubDate/Year").text if article.find(".//PubDate/Year") is not None else "No Date"

        # Abstractをすべて結合
        abstract_parts = [abstract.text for abstract in article.findall(".//AbstractText") if abstract.text]
        abstract = " ".join(abstract_parts) if abstract_parts else "No Abstract"

        # PubMedリンク
        link = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"

        # DOIの取得
        doi = "No DOI"
        elocation_id = article.find(".//ELocationID[@EIdType='doi']")
        if elocation_id is not None:
            doi = elocation_id.text

        articles.append({
            "pmid": pmid,
            "doi": doi,
            "title": title,
            "first_author": first_author,
            "affiliation": affiliation,
            "authors": author_list,
            "journal": journal,
            "pub_date": pub_date,
            "abstract": abstract,
            "link": link,
        })
    return articles


def highlight_keywords(text, keywords):
    """テキスト内のキーワードを赤文字で強調"""
    for keyword in keywords:
        text = text.replace(keyword, f'<span style="color: red;">{keyword}</span>')
    return text

# HTMLメールフォーマット
def format_html_email(articles):
    """解析したデータをHTMLメール用にフォーマット"""
    keywords = load_keywords() or []  # 保存されたキーワードを取得
    content = """
    <html>
    <body>
        <h2>PubMed新着論文通知</h2>
        <hr>
    """
    for article in articles:
        # タイトルとAbstractでキーワードを強調
        highlighted_title = highlight_keywords(article['title'], keywords)
        highlighted_abstract = highlight_keywords(article['abstract'], keywords)

        content += f"""
        <h3>{highlighted_title}</h3>
        <p><strong>PMID:</strong> <a href="{article['link']}">{article['pmid']}</a></p>
        <p><strong>DOI:</strong> {article['doi']}</p>
        <p><strong>First Author:</strong> {article['first_author']} ({article['affiliation']})</p>
        <p><strong>Authors:</strong> {article['authors']}</p>
        <p><strong>Journal:</strong> {article['journal']} ({article['pub_date']})</p>
        <p><strong>Abstract:</strong> {highlighted_abstract}</p>
        <p><a href="{article['link']}">Read more on PubMed</a></p>
        <hr>
        """
    content += """
    </body>
    </html>
    """
    return content


# メール送信
def send_email(subject, html_content, recipient, sender_email, sender_password):
    msg = MIMEMultipart("alternative")
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = recipient
    html_part = MIMEText(html_content, "html")
    msg.attach(html_part)
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient, msg.as_string())

from openpyxl import Workbook, load_workbook
import os

# 検索結果をExcelファイルに保存
from google.colab import drive
import os

# Google Driveのマウント
drive.mount('/content/drive')

# Excel保存関数の修正
def save_to_excel(articles, sheet_name, filename="PubMed_results.xlsx"):
    """検索結果をExcelファイルに保存"""
    # ファイルが存在するか確認
    if os.path.exists(filename):
        workbook = load_workbook(filename)
    else:
        workbook = Workbook()

    # 指定したシートが存在する場合は削除して再作成
    if sheet_name in workbook.sheetnames:
        del workbook[sheet_name]
    sheet = workbook.create_sheet(sheet_name)

    # ヘッダー行を追加
    sheet.append(["PMID", "DOI", "Title", "First Author", "Affiliation", "Authors", "Journal", "Publication Date", "Abstract", "Link"])

    # データを追加
    for article in articles:
        sheet.append([
            article['pmid'],
            article['doi'],
            article['title'],
            article['first_author'],
            article['affiliation'],
            article['authors'],
            article['journal'],
            article['pub_date'],
            article['abstract'],
            article['link']
        ])

    # デフォルトの空シートを削除（最初の作成時のみ存在）
    if "Sheet" in workbook.sheetnames:
        del workbook["Sheet"]

    # ファイルを保存
    workbook.save(filename)
    print(f"Excelファイルに保存しました: {filename}")



# ボタンがクリックされたときの動作
def auto_execute_or_update():
    keywords = load_keywords()
    if keywords:
        # キーワードが保存されている場合、自動実行
        print("保存済みのキーワードを使用して検索を実行します。")
        execute_search(keywords)
    else:
        # 保存済みキーワードがない場合、デフォルトを使用
        print("保存済みキーワードが見つかりません。デフォルトのキーワードを使用します。")
        default_keywords = ["RNA splicing"]
        save_keywords(default_keywords)
        execute_search(default_keywords)

def execute_search(keywords):
    with output:
        output.clear_output()
        print(f"使用する検索キーワード: {keywords}")
        start_date = (datetime.now() - timedelta(days=1)).strftime("%Y/%m/%d")
        end_date = datetime.now().strftime("%Y/%m/%d")

        # 各キーワードについて検索とExcel保存を実行
        for keyword in keywords:
            query = keyword.strip()
            print(f"検索クエリ: {query}")
            new_ids = search_pubmed(query, start_date, end_date, api_key)
            if new_ids:
                xml_data = fetch_abstracts(new_ids, api_key)
                articles = parse_pubmed_data(xml_data)

                # Excelに保存
                save_to_excel(articles, sheet_name=query)

                # メール送信
                html_content = format_html_email(articles)
                send_email(
                    subject=f"PubMed新着論文通知 - {query}",
                    html_content=html_content,
                    recipient=recipient,
                    sender_email=sender_email,
                    sender_password=sender_password
                )
                print(f"メール送信完了: {query}")
            else:
                print(f"{query} に新しい論文はありません。")

# 更新ボタンがクリックされた場合の処理
def on_update_button_click(b):
    with output:
        output.clear_output()
        if text.value.strip():
            new_keywords = text.value.split(",")
            save_keywords(new_keywords)
            print(f"新しいキーワードを保存しました: {new_keywords}")
            execute_search(new_keywords)
        else:
            print("キーワードが入力されていません。")

# 初回実行時の自動処理
print("初期処理を開始します...")
auto_execute_or_update()

# 更新ウィジェットの表示
button.on_click(on_update_button_click)
display(text, button, output)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
初期処理を開始します...
保存済みのキーワードを使用して検索を実行します。


Text(value='', description='キーワード:')

Button(description='検索実行', style=ButtonStyle())

Output()